# POC semantic search E2E analysis

The goal of this POC notebook is to import a model from hugging face, apply it to the dataset, evaluate on metrics, benchmark and create some visualisations. Essentially make sure the semantic search is working end to end.

In [15]:
import plotly.express as px
import pandas as pd
import numpy as np
from ranx import Qrels, Run, evaluate
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity
import random

In [3]:
df_examples = pd.read_parquet('../data/shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet('../data/shopping_queries_dataset_products.parquet')
df_sources = pd.read_csv("../data/shopping_queries_dataset_sources.csv")

In [4]:
# https://github.com/amazon-science/esci-data: suggested filter for task 1: Query-Product Ranking 
# Query-Product Ranking: Given a user specified query and a list of matched products, the goal of this 
# task is to rank the products so that the relevant products are ranked above the non-relevant ones.
df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)

df_task_1 = df_examples_products[df_examples_products["small_version"] == 1]
df_task_1_train = df_task_1[df_task_1["split"] == "train"]
df_task_1_test = df_task_1[df_task_1["split"] == "test"]

In [59]:
# use code provided on huggingface to get started on multilingual text 
# semantic search siamese bert

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
# most basic: TF-IDF
# BM25: bm25 = BM25Okapi(tokenized_docs)
# try siamese BERT: SeyedAli/Multilingual-Text-Semantic-Search-Siamese-BERT-V1
# try sentence-transformers/paraphrase-MiniLM-L6-v2 - sentence-BERT
# try contextualised late interaction over BERT:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = ColBERT.from_pretrained('bert-base-uncased')


df_task_1_train_light = df_task_1_train[df_task_1_train['query_id'].isin([1,2,3,4])]

query_embeddings = model.encode(df_task_1_train_light['query'].tolist(), convert_to_tensor=True)
example_embeddings = model.encode(df_task_1_train_light['product_title'].tolist(), convert_to_tensor=True)

c:\Users\ellen\anaconda3\envs\venv-semantic-search\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ellen\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not in

In [60]:
# calculate cosine similarity and get diagonal
similarities_mx = cosine_similarity(np.array(query_embeddings), np.array(example_embeddings))
similarities_diag = np.diag(similarities_mx)

C:\Users\ellen\AppData\Local\Temp\ipykernel_11768\2807857632.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  similarities_mx = cosine_similarity(np.array(query_embeddings), np.array(example_embeddings))


In [61]:
# apply esci mapping to esci label
esci_weighting = {
    'E': 3,
    'S': 2,
    'C': 1,
    'I': 0
}

df_task_1_train_light['relevance'] = df_task_1_train_light['esci_label'].map(esci_weighting)

C:\Users\ellen\AppData\Local\Temp\ipykernel_11768\831736960.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_task_1_train_light['relevance'] = df_task_1_train_light['esci_label'].map(esci_weighting)


In [62]:
# update index of subset df
df_task_1_train_light = df_task_1_train_light.reset_index(drop=True)
df_task_1_train_light.index = df_task_1_train_light.index + 1

In [63]:
qrels_dict = {}
run_dict = {}
top_n = 5

for query_id, group in df_task_1_train_light.groupby("query_id"):
    query_id_str = str(query_id)
    # get actuals
    qrels_dict[query_id_str] = {str(example): int(relevance) for example, relevance in zip(group["example_id"], group["relevance"])}
    
    # get scores paired to each example
    examples = group["example_id"].tolist()
    example_score_pairs = list(zip(examples, similarities_diag[:len(examples)]))
    
    # filter for top_n examples per query
    example_score_pairs_top_k = sorted(example_score_pairs, key=lambda x: x[1], reverse=True)[:top_n]

    # get predicted
    run_dict[query_id_str] = {str(example): score for example, score in example_score_pairs}
    
    print(f"Query ID: {query_id}")
    for example, score in example_score_pairs_top_k:
        print(f"{score:.2f}, example id {example}")

qrels = Qrels(qrels_dict)
run = Run(run_dict)

results = evaluate(qrels, run, metrics=["ndcg@10"])
print(results)

Query ID: 1
0.67, example id 16
0.62, example id 22
0.60, example id 24
0.59, example id 23
0.59, example id 28
Query ID: 3
0.73, example id 103
0.67, example id 70
0.64, example id 89
0.62, example id 101
0.62, example id 90
0.5896747418894506
